# 超参数搜索

## 什么是超参数？
    在模型训练过程中不变（预设）的参数。比如模型结构参数（层数，激活函数），训练参数（batch_size，学习率，学习率衰减算法）等。

## 什么是超参数搜索？
    设置不同的超参数会得到不一样的训练效果，那么如何得到最好的超参数呢？这种得到最好的超参数组合的方法就是超参数搜索。
    主要有四种方法：网格搜索，随机搜索，，启发式搜索

    1 网格搜索：对于各个超参数，选择性地列出一些可能的取值。将这些值进行组合，对于每一个组合，训练一次，选择效果最好的组合。
            但是这种方法可能会得不到最好的超参数组合，因为超参数的取值是认为设定的。
    2 随即搜索：参数不是手动设定的，而是给定一个范围随机生成的。这种方法的搜索空间更大，有可能得到最优的参数，但是也需要搜索多次。
    3 遗传算法：来自于对自然界的模拟。
            设定一些组数的初始超参数，进行训练，得到生存概率（表现越好生存概率越高）
            A选择：从预训练的参数组中选择一定的组数
            B交叉：将选择特定组数的部分参数进行结合
            C变异：将某些参数进行一些调整
            D产生二代参数组，再与别的二代参数组进行同样操作。
    4 启发式搜索：是研究热点AutoML的一部分，neural architecture search（神经元结构搜索做的就是启发式搜索）
            启发式搜索是使用循环神经网络来生成参数，再使用强化学习来生成反馈，最后使用模型来训练生成参数。
            
## 如何使用sklearn的超参数搜索？
    （1）定义keras模型函数（将要搜索的超参数作为函数的参数），使用KerasRegressor类将keras模型转换为sklearn模型。
    （2）以字典的形式构建超参数的分布（注意字典中代表参数的‘key’要与1中函数的参数相同）
    （3）调用sklearn.model_selction中的Randomizedsearchcv制作一个模型训练器（可以这么理解？），
        进行训练，从村联结果中选择最好模型，保存下来





In [ ]:
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import RandomizedSearchCV

# 准备数据
houses = fetch_california_housing()
# print(houses.data.shape)
# print(houses.target.shape)
# print(houses.DESCR)

x_train_all, x_test_raw, y_train_all, y_test = train_test_split(houses.data, houses.target, random_state=1)
x_train_raw, x_valid_raw, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=2)

# for i in [x_train_raw, x_valid_raw, x_test_raw]:
#     print(i.shape)

ss = StandardScaler()
x_train = ss.fit_transform(x_train_raw)
x_valid = ss.transform(x_valid_raw)
x_test = ss.transform(x_test_raw)

In [ ]:
#（1）
def kr_model(hidden_layers = 3, layer_size = 30, learn_rate = 0.01):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units=layer_size, activation='relu',input_shape = x_train.shape[1:]))
    for i in range(hidden_layers-1):
        model.add(keras.layers.Dense(units=layer_size, activation='relu',input_shape = x_train.shape[1:]))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learn_rate)
    #需要在外部自定义optimizer，然后在optimizer中设置learn_rate
    model.compile(loss = 'mse', optimizer=optimizer)
    return model

sk_model = KerasRegressor(build_fn = kr_model)
sk_model.fit(x_train, y_train,
             validation_data=(x_valid, y_valid),
             epochs = 11,
             )

In [ ]:
#（2）
para_range = {'hidden_layers':[1, 2, 3, 4],
              'layer_size':np.arange(1,100),
              'learn_rate':reciprocal(1e-4,1e-2),}
#这个para_range中的参数要与keras模型函数中的参数相同，因为最后调用的还是自己构建keras模型函数。



In [ ]:
#（3）
random_search_cv = RandomizedSearchCV(sk_model, para_range,
                                      n_iter=10, n_jobs = 1, cv = 3)
#n_iter:保存的参数组的组数  n_jobs:并行训练的模型个数
#cross_validation机制：将原来的训练集分为n份，其中n-1份用来训练，1份当做验证集，cv就是用来指定n的值。
random_search_cv.fit(x_train, y_train,
                     validation_data = (x_valid, y_valid),
                     batch_size = 500, epochs = 10,
                     callbacks = [keras.callbacks.EarlyStopping(patience=3, min_delta=1e-3 )])

#
print(random_search_cv.best_params_) #输出最好的参数
print(random_search_cv.best_score_)  #输出最好的得分
print(random_search_cv.best_estimator_) #输出最好的模型

model = random_search_cv.best_estimator_.model #调用最好的模型
model.evaluate(x_test, y_test)